In [180]:
import instructor
from groq import Groq
from pydantic import BaseModel
import os
from dotenv import load_dotenv
from typing import Dict, Tuple, List
import requests
from litellm import completion
import psycopg2.pool as pool


In [178]:
# loading env
load_dotenv()
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
nessie_key = os.environ.get("NESSIE_API_KEY")
connection_string = os.getenv('DATABASE_URL')

In [139]:
#data types
class DataGeneration(BaseModel):
    income: float
    credit_score: float
    stocks_owned: List[str]
    annual_income: float
    recent_purchases: List[str]
    extra_comments: str
    feedback: str
    experience: int
class UserGeneration(BaseModel):
    first_name: str
    last_name: str
    street_number:str
    street_name:str
    city:str
    state:str
    zip:str
    
    

In [153]:
#Finance data gen prompt
prompt = """I need to generate financial data. I need you to generate annual income which will be a value between 40k-300k(can be a decimal too), I want you to generate a number for credit score, I need you to generate a list of stocks owned, I need you to generate some extra comments after a user's experience talking to an AI agent which is asking financial questions to get information, I need you to generate extra comments, I need you to generate some feedback regadring the same. I need you to generate a number between 1 and 5 for experience. Generate random data because you are a bot that is generating synthetic data, try not to repeat and Return it as a JSON object. Here is an example {'income': 127850.23,
  'credit_score': 720.0,
  'stocks_owned': ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'FB'],
  'annual_income': 127850.23,
  'recent_purchases': ['MSFT', 'AMZN', 'TSLA', 'AAPL'],
  'extra_comments': 'The chatbot was quite helpful in guiding me through the financial questions. The interaction was smooth and the questions were relevant.',
  'feedback': 'Overall, the experience was positive. The AI agent was knowledgeable and the conversation was informative.',
  'experience': 4}"""


In [141]:
prompt2 = """I need to generate general info like name address and all which should be random and should not repeat. Please generate a user with the following information:
                - first_name
                - last_name
                - street_number
                - street_name
                - city
                - state
                - zip
                
                Return it as a JSON object."""

In [142]:
#client model
# client = instructor.from_groq(client, mode=instructor.Mode.TOOLS)
client = instructor.from_litellm(completion, mode =instructor.Mode.MD_JSON)



In [ ]:
# finance data gen
finance_data=[]
i =0
while i<50:

    response = client.chat.completions.create(
        model="groq/llama3-8b-8192",
        max_tokens=1024,
        messages=[{ "role": "user","content": prompt,}],
        response_model=DataGeneration,
    )
    temp_data ={"income":response.income,"credit_score":response.credit_score,"stocks_owned":response.stocks_owned,"annual_income":response.annual_income,
                "recent_purchases":response.recent_purchases,"extra_comments":response.extra_comments,"feedback":response.feedback,"experience":response.experience}
    print(temp_data)
    i+=1
    finance_data.append(temp_data)
    
    


In [ ]:
finance_data

In [ ]:
#user data gen
count =0
url = f"http://api.nessieisreal.com/customers?key={nessie_key}"
user_data=[]
i =0
while count<50:
    response2 = client.chat.completions.create(
        model="groq/llama3-8b-8192",
        max_tokens=1024,
        messages=[{ "role": "user","content": prompt2,}],
        response_model=UserGeneration,
    )
    customer_data = {
    "first_name": response2.first_name,
    "last_name": response2.last_name,
    "address": {
        "street_number": response2.street_number,
        "street_name": response2.street_name,
        "city": response2.city,
        "state": response2.state,
        "zip": response2.zip
    }
        }
    user_data.append(customer_data)
    print(customer_data)
    headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
        }

# Make the POST request
    response = requests.post(url, json=customer_data, headers=headers)

# Check if the request was successful
    if response.status_code == 201:
        print("Customer created successfully!")
        count+=1
    
        
    i+=1
    
    

In [175]:
url = f"http://api.nessieisreal.com/customers?key={nessie_key}"
headers = {
    "Accept": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers)
customers = response.json()
print(len(customers))
# for customer in customers:
#     print(f"Customer ID: {customer['_id']}")
#     print(f"First Name: {customer['first_name']}")
#     print(f"Last Name: {customer['last_name']}")
#     print(f"Address: {customer['address']['street_number']} {customer['address']['street_name']}, {customer['address']['city']}, {customer['address']['state']} {customer['address']['zip']}")
#     print("-" * 40)  # Separator for readability




50


In [177]:
# implementing Postgress through NEON

In [189]:
#create connection pool and establish connection
connection_pool = pool.SimpleConnectionPool(
    1,  # Minimum number of connections in the pool
    10,  # Maximum number of connections in the pool
    connection_string
)
if connection_pool:
    print("Connection pool created successfully")
else:
    print("connection pool missing")
conn = connection_pool.getconn()
cur = conn.cursor()

Connection pool created successfully


In [ ]:
create_table_query = """
CREATE TABLE customer_data (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL,
    street_number VARCHAR(255) NOT NULL,
    street_name VARCHAR(255) NOT NULL,
    city VARCHAR(255) NOT NULL,
    state VARCHAR(255) NOT NULL,
    zip VARCHAR(255) NOT NULL,
    income FLOAT NOT NULL,
    credit_score FLOAT NOT NULL,
    stocks_owned JSONB NOT NULL,
    annual_income FLOAT NOT NULL,
    recent_purchases JSONB NOT NULL,
    extra_comments TEXT,
    feedback TEXT,
    experience INT NOT NULL
);
"""

# Execute the SQL query
cur.execute(create_table_query)
conn.commit()



In [190]:
connection_pool.closeall()
